<a href="https://colab.research.google.com/github/devv-shayan/RAG-APPLICATION-/blob/main/github_rag_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **0. Install required libraries**





In [ ]:
!pip install --upgrade langchain langchain_community langchain-google-genai faiss-cpu sentence-transformers


# **1. Load GitHub Repository Documents**

1.   List item
2.   List item


First, load markdown documents from a specified GitHub repository. This example uses the panaversity/learn-applied-generative-ai-fundamentals repository, but you can adjust it for your own.

In [18]:
from langchain.document_loaders import GithubFileLoader
from google.colab import userdata
import google.generativeai as genai

GITHUB_ACCESS_TOKEN = userdata.get("GITHUB_ACCESS_TOKEN")
genai.configure(api_key=GITHUB_ACCESS_TOKEN)

In [19]:
loader = GithubFileLoader(
    repo="panaversity/learn-applied-generative-ai-fundamentals",
    branch="main",
    access_token=GITHUB_ACCESS_TOKEN,  # Replace with your GitHub access token
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(".md")
)

docs = loader.load()
print(f"Loaded {len(docs)} documents.")

Loaded 128 documents.


# **2. Split Documents into Chunks**
Use the SpacyTextSplitter or RecursiveCharacterTextSplitter to divide the documents into smaller chunks.

 **SpacyTextSplitter**

In [ ]:
from langchain.text_splitter import SpacyTextSplitter

# Install spaCy and download the English model
!pip install spacy
!python -m spacy download en_core_web_sm

# Initialize the semantic text splitter
text_splitter = SpacyTextSplitter(chunk_size=1000, chunk_overlap=200, pipeline='en_core_web_sm')

# Split the documents
splits = text_splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks using SpacyTextSplitter.")


**Using RecursiveCharacterTextSplitter**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(f"Split into {len(splits)} chunks.")

Split into 889 chunks.


In [ ]:
print(f"Chunk: {splits[0].page_content}")

Chunk: # Prologue

## The $100 Trillion Opportunity

**

[Watch: The $100 Trillion AI Industrial Revolution by NVIDIA's Jensen Huang](https://www.youtube.com/watch?v=e5Zol4RYq2o)**

**[From AlexNet to AI Factories: Nvidia's Vision for a $100 Trillion Generative AI Economy](https://www.linkedin.com/pulse/from-alexnet-ai-factories-nvidias-vision-100-trillion-schwentker-bgt7c/)**

**Chip-maker Nvidia became the world's most valuable company after its share price climbed to an all-time high on June 19, 2024.

It is now worth $3.34tn (£2.63tn), with the price having nearly doubled since the start of this year.

**

[AI frenzy makes Nvidia the world's most valuable company](https://www.bbc.com/news/articles/cyrr40x0z2mo)

!

[nvidia price](nvidia.png "nvidia price")

!

[top ten price](topten.png "top ten price")

Nvidia is ushering in a new industrial revolution driven by generative AI - a fundamentally new way of computing.


# **3. Create Embeddings using Sentence Transformers**
Generate semantic embeddings for each document chunk using the all-MiniLM-L6-v2 model.

In [6]:
from langchain.embeddings import SentenceTransformerEmbeddings

In [ ]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print("Embeddings created successfully.")

In [ ]:
print(f"Embedding dimension: {embeddings}")

Embedding dimension: client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


# **4. Store Embeddings in FAISS**
Store the embeddings in a FAISS vector store for efficient document retrieval.

In [9]:
from langchain.vectorstores import FAISS


In [10]:
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
print(f"Vector store created with {vectorstore.index.ntotal} embeddings.")
print("Vector store created using FAISS.")

Vector store created with 919 embeddings.
Vector store created using FAISS.


# **5. Initialize a Retriever**
Use the FAISS vector store to retrieve relevant documents based on semantic similarity.

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}  # Retrieve top 5 similar documents
)

# Retrieve relevant documents
# retrieved_docs = retriever.invoke("why we are using langchain ?")
# print(f"Retrieved {len(retrieved_docs)} documents.")
# print(retrieved_docs)

# **6. Initialize the Google Gemini Language Model**
Configure the Google Gemini AI to generate language model responses.

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [15]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GEMINI_API_KEY # Replace with your Google API key
)
print("LLM initialized.")

LLM initialized.


# **7. Create the RAG Chain**
Pull a pre-defined prompt from the LangChain hub and create the RAG chain that ties the document retrieval with LLM responses.

In [28]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

In [24]:
# Pull prompt from LangChain hub
prompt = hub.pull("rlm/rag-prompt")

# Create RAG Chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


# **8. Run the RAG Chain**
Ask a question, and the RAG chain will retrieve relevant documents and generate an answer using the language model.

In [25]:
response = rag_chain.invoke("how to make briyani?")
print("Response:")
print(response)

Response:
I'm sorry, but the context provided does not contain information about how to make biryani. The context focuses on AI-driven instructions for cooking, but it does not include any recipes or instructions for biryani. 



# **Conclusion**
This project demonstrates a scalable Retrieval-Augmented Generation (RAG) pipeline using LangChain, FAISS, and Google Gemini AI. It enables document retrieval from a GitHub repository and uses an LLM to generate responses based on relevant retrieved content.